In [1]:
# Applying kcorrect_python to SDSS observed ugriz data
# Download
# 1. kcorrect package: http://cosmo.nyu.edu/blanton/kcorrect/kcorrect.v4_3.tar.gz
# 2. kocrrect_python: https://pypi.python.org/pypi/kcorrect_python/2017.07.05
# Installation
# 1. see kcorrect install
# 2. see kocrrect_python readme

# Some physics links
# Measures of flux and measurements: http://www.sdss.org/dr12/algorithms/magnitudes/
# kcorrect: http://kcorrect.org

Given the excellent agreement between cmodel magnitudes (see cmodel magnitudes above) and PSF magnitudes for point sources, and between cmodel magnitudes and Petrosian magnitudes (albeit with intrinsic offsets due to aperture corrections) for galaxies, the cmodel magnitude is now an adequate proxy to use as a universal magnitude for all types of objects. As it is approximately a matched aperture to a galaxy, it has the great advantage over Petrosian magnitudes, in particular, of having close to optimal noise properties.

A "maggy" is the flux f of the source relative to the standard source f0 (which defines the zeropoint of the magnitude scale). Therefore, a "nanomaggy" is 10^-9 times a maggy. 
To relate these quantities to standard magnitudes, an object with flux f given in nMgy has a Progson magnitude:
m = [22.5 mag] - 2.5log10f
- Note that magnitudes listed in the SDSS catalog, however, are not standard Pogson magnitudes, but asinh magnitudes.
- Magnitudes within the SDSS are expressed as inverse hyperbolic sine (asinh) magnitudes, sometimes referred to informally as luptitudes. 

The relation between detected flux f and asinh magnitude m is:
    m = -2.5 / ln(10) * [asinh((f/f0) / (2b)) + ln(b)]

Here f0 is given by the classical zero point of the magnitude scale, i.e., f0 is the flux of an object with conventional magnitude of zero.

Asinh softening parameters

| Filter | b      | zero-flux magnitude [m(f/f0 = 0)] | m(f/f0 = 10b) |
|:-------|:------:|:-------:|:--------:|
|  u     | 1.4e-10| 24.63 | 22.12 |
|  g     | 0.9e-10| 25.11 | 22.60 |
|  r     | 1.2e-10| 24.80 | 22.29 |
|  i     | 1.8e-10| 24.36 | 21.85 |
|  z     | 7.4e-10| 22.83 | 20.32 |


SDSS ugriz magnitudes to AB ugriz magnitudes
- u: u_AB = u_SDSS - 0.04 mag
- g,r,i are close to AB
- z: z_AB = z_SDSS + 0.02 mag

## maginitudes to maggy
1. cmodelmag - extinxtion_correction
2. SDSS to AB
3. asinh mag to flux
   f = sinh([m / (-2.5/ln(10)) - ln(b)]) * (2b) * f0

## maggy_ivar
Note that the conversion to the inverse variances from the maggies and the magnitude errors is (0.4 ln(10) × maggies × magerr)-2

what are the magerr? Can be downloaded from SDSS skyserver

In [2]:
import kcorrect as kc
import numpy as np
import os
import pandas as pd

In [42]:
# load the unLRG sample list
listpath = "./BH_SDSS_cross_checked.xlsx"
data = pd.read_excel(listpath, "Sheet2")

In [97]:
def calc_maggies(mag, ext, band_id):
    """Calc flux from magnitude"""
    # extinction correction
    mag = mag - ext
    # AB calibration
    ab_coeff = [-0.04, 0, 0, 0, 0.02]
    mag = mag + mag * ab_coeff[band_id]
    # mag to maggie
    f0 = 3631 #[Jy]
    b_coeff = [1.4e-10, 0.9e-10, 1.2e-10, 1.8e-10, 7.4e-10]
    b = b_coeff[band_id]
    # maggie = math.sinh((mag / (-2.5/math.log(10)) - math.log(b))) * (2*b) * f0
    maggie = 10 ** (mag / -2.5)
    
    return maggie

In [24]:
# TODO
def calc_maggies_ivar(maggie, magerr):
    "Calc maggie inverse variance"
    maggie_ivar = (0.4 * math.log(10) * maggie * magerr)**(-2)
    return maggie_ivar

In [53]:
def get_sample_params(mags,exts,magerrs,z):
    """Generate the parameters for kcorrection estimation"""
    param = np.zeros((11,))
    param[0] = z
    # calc maggies
    for i, mag in enumerate(mags):
        param[i+1] = calc_maggies(mag=mag, ext=exts[i], band_id=i)
    # calc maggie_ivar
    for i, magerr in enumerate(magerrs):
        param[i+6] = calc_maggies_ivar(maggie=param[i+1], magerr=magerr)
    
    return param

In [26]:
# calc reconstructed_mag
def calc_reconmag(sample_params):
    kc.load_templates() # load the default template
    kc.load_filters() # load the SDSS filters
    # get the coeffs
    coeff = kc.fit_coeffs(sample_params)
    reconmag = kc.reconstruct_maggies(coeff)
    return reconmag

In [74]:
# calc the absolute magnitude from magnitude_r and k_correction
from astropy.cosmology import FlatLambdaCDM
import astropy.units as au
def calc_luminosity_distance(redshift):
    """Calculate the rate, kpc/px."""
    # Init
    # Hubble constant at z = 0
    H0 = 71.0
    # Omega0, total matter density
    Om0 = 0.27
    # Cosmo
    cosmo = FlatLambdaCDM(H0=H0, Om0=Om0)
    # Angular diameter distance, [Mpc]
    dL = cosmo.luminosity_distance(redshift)

    return dL.to(au.pc)

def calc_absmag(mag_r,dl,kcrt):
    mag_abs = mag_r - 5*(math.log10(dl) - 1) - kcrt
    return mag_abs

## Single test

In [43]:
data.keys()

Index(['Name-J2000', 'RA', 'Dec', 'z', 'raj2000', 'dej2000', 'S_nvss',
       'cmodelmag_g', 'cmodelmag_i', 'cmodelmag_r', 'cmodelmag_u',
       'cmodelmag_z', 'extinction_g', 'extinction_i', 'extinction_r',
       'extinction_u', 'extinction_z', 'cmodelmagerr_g', 'cmodelmagerr_i',
       'cmodelmagerr_r', 'cmodelmagerr_u', 'cmodelmagerr_z', 'Type'],
      dtype='object')

In [44]:
# a single test
extinction_u = data["extinction_u"]
extinction_g = data["extinction_g"]
extinction_r = data["extinction_r"]
extinction_i = data["extinction_i"]
extinction_z = data["extinction_z"]

cmodelmag_u = np.nan_to_num(data["cmodelmag_u"])
cmodelmag_g = np.nan_to_num(data["cmodelmag_g"])
cmodelmag_r = np.nan_to_num(data["cmodelmag_r"])
cmodelmag_i = np.nan_to_num(data["cmodelmag_i"])
cmodelmag_z = np.nan_to_num(data["cmodelmag_z"])

cmodelmagerr_u = np.nan_to_num(data["cmodelmagerr_u"])
cmodelmagerr_g = np.nan_to_num(data["cmodelmagerr_g"])
cmodelmagerr_r = np.nan_to_num(data["cmodelmagerr_r"])
cmodelmagerr_i = np.nan_to_num(data["cmodelmagerr_i"])
cmodelmagerr_z = np.nan_to_num(data["cmodelmagerr_z"])

In [31]:
idx_u1 = np.where(cmodelmag_u != -9999)[0]
idx_u2 = np.where(cmodelmag_u != 0.0)[0]
idx_u3 = np.where(cmodelmag_u != 10000)[0]
idx = np.intersect1d(idx_u1,idx_u2)
idx = np.intersect1d(idx, idx_u3)

In [37]:
redshift = data["z"]

In [110]:
i = 1109
z = redshift[i]
mags = [cmodelmag_u[i],cmodelmag_g[i],cmodelmag_r[i],cmodelmag_r[i],cmodelmag_z[i]]
exts = [extinction_u[i],extinction_g[i],extinction_r[i],extinction_r[i],extinction_z[i]]
magerrs = [cmodelmagerr_u[i],cmodelmagerr_g[i],cmodelmagerr_g[i],cmodelmagerr_i[i],cmodelmagerr_z[i]]
params = get_sample_params(mags,exts,magerrs,z)

In [111]:
params

array([  2.84619987e-01,   3.30770987e-08,   7.98178393e-08,
         2.01748669e-07,   2.01748669e-07,   3.03189810e-07,
         2.15039387e+17,   3.09703568e+18,   4.84757777e+17,
         1.14866669e+18,   1.12387971e+17])

In [112]:
reconmag = calc_reconmag(params)

In [113]:
reconmag

array([  2.84619987e-01,   2.81977819e-08,   8.14878121e-08,
         1.83831446e-07,   2.10137159e-07,   2.76834669e-07], dtype=float32)

In [114]:
dl = calc_luminosity_distance(z)
# mag_abs = calc_abmag(mags[2],dl,reconmag[3])

In [117]:
dl

<Quantity 1449904937.1746511 pc>

In [118]:
5*(math.log10(dl.value) - 1)

40.80669764354079

In [119]:
mags[2]

16.898117065429702

In [120]:
reconmag[3]

1.8383145e-07

In [121]:
mags[2] - 40.8 - reconmag[3]

-23.901883118401742

In [122]:
mag_abs = calc_absmag(mags[2],dl.value,reconmag[3])

In [123]:
mag_abs

-23.908580761942531

In [106]:
f = 10 ** (mag_abs/(-2.5))

In [107]:
f

976.91973992496946